In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
wd = webdriver.Chrome("./chromedriver.exe") 
wd.implicitly_wait(10)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4292\3653437403.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome("./chromedriver.exe")


In [3]:
wd.get("https://coinmarketcap.com/")


In [4]:
content = wd.page_source.encode("UTF-8").strip()

In [5]:
soup = BeautifulSoup(content, "html.parser")

In [6]:
names = []
prices = []
daily_changes = []
weekly_changes = []

#Get all currencies + fill the names list
coins = soup.select("tbody tr")
for i in range(10):
 coin = coins[i].select("p")
 names.append([coin[1].get_text(), coin[2].get_text()])

 

In [8]:
#Get all prices + prices changes
for i in range(10):
 coin = coins[i].select("span")
 prices.append(coin[2].get_text())
 daily_changes.append([coin[5].get_text(), coin[5].find_next("span")["class"][0]])
 weekly_changes.append([coin[7].get_text(), coin[7].find_next("span")["class"][0]])

tbody = """"""

for i in range(10):
 tbody += f"""
 <tr>
  <td>
   {i+1}
  </td>
  <td>
   {names[i][0] + " | " + names[i][1]}
  </td>
  <td>
   {prices[i]}
  </td>
  <td>
   {("U" if daily_changes[i][1] == "icon-Caret-up" else "D") +  daily_changes[i][0]}
  </td>
  <td>
   {("U" if weekly_changes[i][1] == "icon-Caret-up" else "D") + weekly_changes[i][0]}
  </td>
 </tr>
  \n"""


 <tr>
  <td>
   1
  </td>
  <td>
   Bitcoin | BTC
  </td>
  <td>
   $24,036.62
  </td>
  <td>
   U5.38%
  </td>
  <td>
   U4.45%
  </td>
 </tr>
  

 <tr>
  <td>
   2
  </td>
  <td>
   Ethereum | ETH
  </td>
  <td>
   $1,872.08
  </td>
  <td>
   U12.05%
  </td>
  <td>
   U14.28%
  </td>
 </tr>
  

 <tr>
  <td>
   3
  </td>
  <td>
   Tether | USDT
  </td>
  <td>
   $1.00
  </td>
  <td>
   U0.01%
  </td>
  <td>
   U0.01%
  </td>
 </tr>
  

 <tr>
  <td>
   4
  </td>
  <td>
   USD Coin | USDC
  </td>
  <td>
   $1.00
  </td>
  <td>
   U0.02%
  </td>
  <td>
   D0.03%
  </td>
 </tr>
  

 <tr>
  <td>
   5
  </td>
  <td>
   BNB | BNB
  </td>
  <td>
   $331.78
  </td>
  <td>
   U3.94%
  </td>
  <td>
   U10.35%
  </td>
 </tr>
  

 <tr>
  <td>
   6
  </td>
  <td>
   XRP | XRP
  </td>
  <td>
   $0.3831
  </td>
  <td>
   U5.36%
  </td>
  <td>
   U2.76%
  </td>
 </tr>
  

 <tr>
  <td>
   7
  </td>
  <td>
   Cardano | ADA
  </td>
  <td>
   $0.5451
  </td>
  <td>
   U7.87%
  </td>
  <td>
   U7.83%
  </

In [9]:
#SMTPLIB email sender
port = 587
smtp_server = "smtp.gmail.com"
sender_email = "WHAT DO YOU"
receiver_email = "WANT TO"
password = "FIND HERE?"

message = MIMEMultipart("message")
message["Subject"] = "UpTime Update"
message["From"] = sender_email
message["To"] = receiver_email

html_message = f"""\
 <html>
  <body>
   {tbody}
  </body>
 </html>
 """

message_object = MIMEText(html_message, "html")

message.attach(message_object)


 <tr>
  <td>
   1
  </td>
  <td>
   Bitcoin | BTC
  </td>
  <td>
   $24,036.62
  </td>
  <td>
   U5.38%
  </td>
  <td>
   U4.45%
  </td>
 </tr>
  

 <tr>
  <td>
   2
  </td>
  <td>
   Ethereum | ETH
  </td>
  <td>
   $1,872.08
  </td>
  <td>
   U12.05%
  </td>
  <td>
   U14.28%
  </td>
 </tr>
  

 <tr>
  <td>
   3
  </td>
  <td>
   Tether | USDT
  </td>
  <td>
   $1.00
  </td>
  <td>
   U0.01%
  </td>
  <td>
   U0.01%
  </td>
 </tr>
  

 <tr>
  <td>
   4
  </td>
  <td>
   USD Coin | USDC
  </td>
  <td>
   $1.00
  </td>
  <td>
   U0.02%
  </td>
  <td>
   D0.03%
  </td>
 </tr>
  

 <tr>
  <td>
   5
  </td>
  <td>
   BNB | BNB
  </td>
  <td>
   $331.78
  </td>
  <td>
   U3.94%
  </td>
  <td>
   U10.35%
  </td>
 </tr>
  

 <tr>
  <td>
   6
  </td>
  <td>
   XRP | XRP
  </td>
  <td>
   $0.3831
  </td>
  <td>
   U5.36%
  </td>
  <td>
   U2.76%
  </td>
 </tr>
  

 <tr>
  <td>
   7
  </td>
  <td>
   Cardano | ADA
  </td>
  <td>
   $0.5451
  </td>
  <td>
   U7.87%
  </td>
  <td>
   U7.83%
  </

In [10]:
context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
 server.ehlo()
 server.starttls(context = context)
 server.ehlo()
 server.login(sender_email, password)
 server.sendmail(sender_email, receiver_email, message.as_string())